In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import fill_values as fv
import pandas as pd
import numpy as np
from scipy.integrate import simps


In [ ]:
def get_value_at_station(val):
    array = pd.DataFrame.from_dict(val[:,:])
#    val_ml = pd.DataFrame.from_dict(val_ml[:,:,ens_memb,y[0],x[0]])
    return(array)

In [ ]:
def get_thickness(surface_air_pressure, 
                  air_temperature_0m, air_temperature_ml,
                  ap, b,
                 ens_memb,y,x,):
    ### Calculuat the thickness of each layer in m or geop height
    
### 1) Connect model levels and surface values
# this is only done for pressure and temperature
# !!!! DO NOT DO THAT FOR THE OTHER VALUES, SINCE in model levels it is the mixing ration and at the surface [kg/m^2]

### Pressure
# transform hybrid sigma pressure coordinate in model levels to actual pressure
    surface_air_pressure, dt_sap = fv.mask_array(surface_air_pressure,ens_memb,y,x)
    p_sfc = get_value_at_station(surface_air_pressure) 
    p_ml = pd.DataFrame.from_dict((ap[:] + b[:]*surface_air_pressure[:,:]))
    pressure = np.asarray(pd.concat([p_ml, p_sfc],axis = 1))

### Temperature
    air_temperature_0m, dt_at0m = fv.mask_array(air_temperature_0m,ens_memb,y,x)
    air_temperature_ml, dt_atml = fv.mask_array(air_temperature_ml,ens_memb,y,x)
    t_sfc = get_value_at_station(air_temperature_0m)
    t_ml = get_value_at_station(air_temperature_ml)
    temperature = np.asarray(pd.concat([t_ml,t_sfc],axis = 1))

### 2) to convert pressure-levels into actual heights use the hypsometric equation --> Temperature and pressure
# are needed. After J. E. Martin: Mid-Latitude Atmospheric Dynamics Eq. 3.6
    Rd = 287.    # gas constant for dry air [J kg^-1 K^-1]
    g = 9.81     # Standard gravity [m s^-2]

### calculate the pressure-weighted, column averaged temperature as from J. E. Martin: Mid-Latitude Atmospheric 
# Dynamics Book, Eq. below Eq. 3.6
    temp_mean = []
    for i in range(0, temperature.shape[1]):
        numT = simps(y=temperature[:,i:(i+2)], x=pressure[:,i:(i+2)], dx = np.log(pressure[:,i:(i+2)]))
        denomT = simps(y=np.ones(temperature[:,i:(i+2)].shape), x = pressure[:,i:(i+2)], dx = np.log(pressure[:,i:(i+2)]))
        t_mean = numT/denomT
        temp_mean.append(t_mean)
# get temperature and pressure, and value so that array zero contains low levels (transpose or flip)
    temp_mean = pd.DataFrame.from_dict(np.transpose((temp_mean)))
    temp_mean = np.fliplr(temp_mean)
    pres = np.fliplr(pressure)
    temperature = np.fliplr(temperature)

    thickness = []
    geop_th = []
    for i in range(0, pres.shape[1]):
        if (i+1) == pres.shape[1]:
            continue
        p1 = pres[:,i]
        p2 = pres[:,(i+1)]   
        dz = (Rd * temp_mean[:,i])/g * np.log((p1/p2))    # thickness in [m]
        dgeop = (Rd * temp_mean[:,i])* np.log((p1/p2))    # thickness in [J/kg]
        thickness.append(dz)
        geop_th.append(dgeop)
# transpose array, so that array starts at lower levels
    thickness = pd.DataFrame.from_dict(np.transpose((thickness)))
    geop_th = pd.DataFrame.from_dict(np.transpose((geop_th)))
    
    return(thickness, geop_th)